# PAA - Dark Souls, where are my keys ?

Dark Souls é um jogo eletrônico do gênero RPG (Role Playing Game) que faz parte de uma das séries de jogos mais populares dentro da categoria, a série Souls. Essa série é especialmente conhecida por ter uma dificuldade acentuada. Em especial, o Dark Souls 1, tema deste deste projeto, tem essa dificuldade muito relacionada com o design do mapa. Isso porque os designers que criaram o mapa do jogo modelaram ele a partir de um labirinto, de forma que existem muitas ligações entre as diferentes áreas e não há nenhum tipo de mapa ou bússola que dentro do jogo vá te dizer para onde ir.

Essa característica do mapa faz com que muitos novos jogadores acabem indo parar em áreas de um nível de dificuldade muito superior ao nível do seu personagem, e sem saber como voltar ou se estão no lugar correto, acabam ficando travados e muitos até desistem do jogo. A ideia aqui é prover um algoritmo que calcula o caminho mais fácil para chegar ao destino que o jogador deseja, partindo do ponto onde ele está.

Mais que isso, essa ideia também será usada para indicar ao jogador qual caminho seguir para encontrar certos itens que ele deseja.



### PT. 1 - Modelar o problema

A ideia é construir o mapa do jogo como um grafo, onde os nós são as áreas e as arestas são os caminhos entre elas. 

#### 1.1 Visão superior do mapa

![map](assets/ds-map.png)


#### 1.2 Modelo inicial 

Para construir o grafo mais facilmente, o primeiro passo foi chegar a um modelo que nos mostrasse as relações entre as áreas de jogo e quais são os caminhos que definem essas relações. Isso para que posteriormente possamos também definir a dificuldade de cada caminho.

![map2](assets/ds-fluxo.png)


#### 1.3 Peso das arestas

Da forma como estamos modelando nosso problema o peso da aresta que liga A e B representa a dificuldade de transitar pelo caminho representado por essa aresta. É óbvio concluir que "dificuldade" é muitas vezes um conceito muito subjetivo para ser definido de forma númerica. Mas no caso de um jogo é possível tentar discretizar esse conceito em pontos que normalmente causam ao jogador uma dificuldade maior de realizar alguma ação.

Em nosso caso, os seguintes pontos foram levados em conta:

* **Nível de personagem recomendado para cada área:** essa métrica foi desenvolvida pela própria comunidade do jogo e leva em conta o nível dos inimigos da área, dos chefões, e outros pontos como armadilhas e dificuldade do terreno. Nas referências do projeto podem ser encontradas as fontes desses dados.
![area-levels](assets/area-levels.png)

* **Dificuldade específica do caminho:** Alguns dos caminhos descritos no nosso modelo não são os usuais que os jogadores seguiriam explorando o jogo. Esses na realidade são atalhos, comummente chamados de "skips", que podem te levar de uma área a outra através de formas alternativas. Alguns são pulos bem difíceis de executar, outros são uma porta/passagem que está muito bem escondida ou é protegida por algum inimido especial. Pra esses casos olhar o nível recomendado não basta, então cada aresta que se enquadra nesse caso recebeu uma análise individual.

* **Experiência própria:** Por fim, também levei em conta a minha experiência própria de jogador veterano da série para fazer alguns ajustes 


In [3]:
from collections import deque, namedtuple


# we'll use infinity as a default distance to nodes.
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')


def make_edge(start, end, cost=1):
      return Edge(start, end, cost)


class Graph:
    def __init__(self, edges):
        # let's check that the data is right
        wrong_edges = [i for i in edges if len(i) not in [2, 3]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges]

    @property
    def vertices(self):
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    def remove_edge(self, n1, n2, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                self.edges.remove(edge)

    def add_edge(self, n1, n2, cost=1, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost))

    @property
    def neighbours(self):
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))
            neighbours[edge.end].add((edge.start, edge.cost))
        return neighbours

    def dijkstra(self, source, dest):
        assert source in self.vertices, 'Such source node doesn\'t exist'
        distances = {vertex: inf for vertex in self.vertices}
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        distances[source] = 0
        vertices = self.vertices.copy()

        while vertices:
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path


adjacency = []

    
graph = Graph([
    
    ("Painted World", "Anor Londo", 5),  ("Anor Londo", "Duke's Archives", 7),  
    ("Duke's Archives", "Crystal Caves", 7), ("Anor Londo", "Sen's Fortress", 4),
    ("Sen's Fortress", "Undead Parish", 3), ("Undead Parish", "Darkroot Garden", 3), 
    ("Undead Parish", "Firelink Shrine", 2),  ("Darkroot Garden", "Darkroot Basin", 3), 
    ("Undead Parish", "Upper Burg", 2), ("Tomb of the Giants", "Catacombs", 7),
    ("Catacombs", "Firelink Shrine", 5), ("Firelink Shrine", "Undead Asylum", 2),
    ("Firelink Shrine", "Upper Burg", 2),  ("Firelink Shrine", "Upper Londo", 5),  
    ("Firelink Shrine", "Firelink Altar", 6),  ("Firelink Altar", "The Kiln", 10), 
    ("Firelink Altar", "The Abyss", 7), ("The Abyss", "Lower Londo", 7),
    ("Lower Londo", "Quelaag's Domain", 5), ("Quelaag's Domain", "Upper Demon Ruins", 5), 
    ("Quelaag's Domain", "Mid Demon Ruins", 4), ("Quelaag's Domain", "Lower Demon Ruins", 5),
    ("Upper Demon Ruins", "Mid Demon Ruins", 4), ("Mid Demon Ruins", "Lower Demon Ruins", 5),
    ("Mid Demon Ruins", "Lost Izalith", 6), ("Lower Demon Ruins", "Lost Izalith", 6),
    ("Upper Burg", "Lower Burg", 2), ("Upper Burg", "Darkroot Basin", 3),
    ("Darkroot Basin", "Oolacile Sanctuary Garden", 5), ("Oolacile Sanctuary Garden", "Oolacile Sanctuary", 5),
    ("Oolacile Sanctuary", "Royal Wood", 5), ("Royal Wood", "Oolacile Township", 8),
    ("Oolacile Township", "Chasm of the Abyss", 10), ("Oolacile Township", "Battle of Stoicism", 6),
    ("Darkroot Basin", "Valley of Drakes", 4), ("Upper Londo", "Valley of Drakes", 4),
    ("Valley of Drakes", "Lower Blighttown", 4), ("Lower Blighttown", "Upper Blighttown", 5),
    ("The Depths",  "Upper Blighttown", 4), ("Upper Londo", "The Abyss", 5),
    ("Upper Londo",  "Lower Londo", 7), ("Valley of Drakes", "Lower Londo", 6),
    ("Lower Blighttown",  "Quelaag's Domain", 4), ("Lower Blighttown", "Great Hollow", 5),
    ("Great Hollow", "Ash Lake", 4)
])

itens_location = {
    "Master Key":["The Depths"],
    "Dungeon Cell Key": ["Undead Asylum"],
    "Undead Asylum F2 East Key": ["Undead Asylum"],
    "Undead Asylum F2 West Key": ["Undead Asylum"],
    "Big Pilgrim's Key": ["Undead Asylum"],
    "Residence Key": ["Upper Burg"],
    "Mystery Key": ["Undead Parish"],
    "Basement Key": ["Lower Burg"],
    "Key to Depths": ["Lower Burg"],
    "Sewer Chamber Key": ["The Depths"],
    "Blighttown Key": ["The Depths"],
    "Watchtower Basement Key": ["Darkroot Garden"],
    "Key to New Londo Ruins": ["Blighttown"],
    "Key to the Seal": ["Upper Londo"],
    "Cage Key": ["Sen's Fortress"],
    "Archive Tower Cell Key": ["Duke's Archives"],
    "Archive Prison Extra Key": ["Duke's Archives"],
    "Archive Tower Giant Door Key": ["Duke's Archives"],
    "Archive Tower Giant Cell Key": ["Duke's Archives"],
    "Annex Key": ["Painted World"],
    "Crest of Artorias": ["Darkroot Basin"],
    "Peculiar Doll": ["Undead Asylum"],
    "Broken Pendant": ["Duke's Archives"],
    "Crest Key": ["Oolacile Township"]
}

starting_point = input("In which place are you ?")
desired_item = input("What key do you desire ?")

print(graph.dijkstra(starting_point, itens_location.get(desired_item)[0]))

In which place are you ?123123123123
What key do you desire ?123213132


TypeError: 'NoneType' object is not subscriptable